Me gustaría que el usuario me de:
- Pairings: uno o más combinaciones de alimentos con los que quiere maridar el vino. (OK)
- Body, tannins, sweetness, acidity: de 0 a 1 (OK)
- Uvas: una selección de uvas que yo determine + otras (todas las que no estén incluidas dentro de las que le doy como opción individual). (OK)
- Price: el rango de precios en el cual buscar vinos. (OK)

- Gustó la recomendación: debería haber una función que te diga si al usuario le gustó o no la recomendación en base a diversos parámetros (pensar en el modelo por puntaje y filtro que hice).

1. Pasar a Clase
2. Crear datos sintéticos

In [ ]:
import random
import sys
import os
import logging

import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler

sys.path.append(os.path.abspath(os.path.join('..', '..', 'src', 'utils')))
import utils as ut

### user input

In [2]:
# randint: https://www.w3schools.com/python/ref_random_randint.asp
# Pandas .quantile: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html
# np.random.normal: https://numpy.org/doc/2.1/reference/random/generated/numpy.random.normal.html
# np.clip: https://numpy.org/doc/2.1/reference/generated/numpy.clip.html 
# random.choices: https://docs.python.org/3/library/random.html#random.choices


# Selección de pairing
def select_random_meal(meal_df):
    i = np.random.randint(0, len(meal_df))
    return meal_df.loc[i]

# Elección de categoría según un valor dado
def get_category(value, quantile_values, categories=["leve", "moderado", "marcado", "intenso"]):
    if len(quantile_values)-1 != len(categories):
        raise IndexError("Length of quantiles ranges and categories does not match!")
    
    quant_values = quantile_values.values

    # Obtiene categoría y valor promedio del rango de los cuantiles
    for i in range(0, len(quant_values)-1):
        start_val = quant_values[i]
        end_val = quant_values[i+1]

        if start_val <= value < end_val:
            cat = categories[i]
            val_range = [start_val, end_val]
            return cat, val_range
    
    # Gestiona valores atípicos (mayores o menores que el máximo o mínimo valor de los cuantiles)
    if value >= end_val:
        cat = categories[i] # Último i asignado es el mayor i
        val_range = [start_val, end_val]
        return cat, val_range
    
    if 0 <= value < quant_values[0]:
        cat = categories[0]
        val_range = [quant_values[0], quant_values[1]]
        return cat, val_range
    
    # Levanta error si el valor es inferior a cero
    if value < 0:
        raise ValueError(f"Passed value is less than zero! {value}")

# Creación de perfiles de sabor por pairing (para elecciones que hagan sentido con la elección del user)
def build_pairing_profile(df, pairing_cols, taste_cols, quantiles=[0, .25, .5, .75, 1]):
    profiles = {}
    deviations = {}
    for pairing in pairing_cols:
        pairing_subset = df[df[pairing] == 1]
        profiles[pairing] = {}
        deviations[pairing] = {}
        for taste in taste_cols:
            # Cuantiles
            taste_quantile = pairing_subset[taste].quantile(quantiles)
            profiles[pairing][taste] = taste_quantile
            # Desvío Estandar (para distribución normal en select_taste_profile)
            std = pairing_subset[taste].std()
            deviations[pairing][taste] = std

    return profiles, deviations

# Selección del perfil de vino con algo de sentido común (en general - sumamos probabilidad de random choice)
def select_taste_profile(taste_cols, pairing, profiles, deviations, prob=.2, categories=["leve", "moderado", "marcado", "intenso"]):
    selected_profile = {}
    for taste in taste_cols:
        if random.random() < prob:
            quantiles = profiles[pairing][taste]
            quant_random_options = []
            for i in range(0, len(quantiles.values)-1):
                mid_val = (quantiles.values[i] + quantiles.values[i+1])/2
                quant_random_options.append(mid_val)
            quant_random_choice = np.random.choice(quant_random_options)
            selected_profile[taste] = get_category(quant_random_choice, quantile_values=quantiles, categories=categories)
        else:
            quantiles = profiles[pairing][taste]
            median = quantiles[.5]
            std = deviations[pairing][taste]
            min_val = quantiles[0]
            max_val = quantiles[1]
            valor_seleccionado = min(max(np.random.normal(loc=median, scale=std), min_val), max_val)
            selected_profile[taste] = get_category(valor_seleccionado, quantile_values=quantiles, categories=categories)
    return selected_profile

# Obtención de top N grapes y su cantidad
def get_top_grapes(grapes_df, top_n_grapes):
    top_grapes = grapes_df.sum(axis=0).sort_values(ascending=False).head(top_n_grapes)
    return top_grapes

# Selección de uvas según probabilidad de uva en dataset
def select_grapes(grapes_df, top_n_grapes, min_grapes, max_grapes):

    top_grapes = get_top_grapes(grapes_df, top_n_grapes)
    total_grapes_in_df = sum(grapes_df.sum(axis=0))
    top_grapes_prob = top_grapes / total_grapes_in_df
    top_grapes_prob["Otras Uvas"] = 1 - sum(top_grapes_prob)
    grapes_prob = dict(top_grapes_prob)

    while True:
        grape_selection = []
        for key, value in grapes_prob.items():
            if random.random() < value:
                grape_selection.append(key)

        if len(grape_selection) >= min_grapes:
            break

    if len(grape_selection) > max_grapes:
        grape_selection = grape_selection[:max_grapes]

    return grape_selection


# Selección de rango de precios considerando lógica de usuarios
def select_price_range(df,
                       quantiles=[.075, .125, .25, .375, .5, .625, .75, .875],
                       low_prices_weights=[.2, .3, .25, .15, .05, .025, .015, 0.01, 0, 0],
                       min_dispersion=2):
    
    no_outlier_price = ut.manage_outlier_IQR(df=df["price"], func="remove")
    price_quantiles = no_outlier_price.quantile(quantiles)
    price_quantiles[0], price_quantiles[1] = 0, -1
    price_quantiles = price_quantiles.sort_index()
    price_quantiles = pd.Series(price_quantiles.values)

    low_price_index = random.choices(population=range(len(price_quantiles)), weights=low_prices_weights, k=1)[0]

    high_top_limit = len(price_quantiles)
    high_bottom_limit = low_price_index + min_dispersion if (low_price_index + min_dispersion) < len(price_quantiles) else len(price_quantiles)-1

    high_price_possible = range(high_bottom_limit, high_top_limit)

    high_prices_weights = [1 / x**1.5 for x in high_price_possible]

    high_price_index = random.choices(population=high_price_possible, weights=high_prices_weights, k=1)[0]

    price_range = [int(price_quantiles[low_price_index]), int(price_quantiles[high_price_index])]

    if price_range[1] == -1:
        price_range[1] = None

    return price_range

### elección de vino

In [3]:
# Lógica para determinar fuzzy distance del perfil de vino seleccionado respecto a los vinos disponibles
def fuzzy_distance(value, q_low, q_high):
    if value < q_low:
        return (q_low - value) / (q_high - q_low)
    elif value > q_high:
        return (value - q_high) / (q_high - q_low)
    else:
        return 0
    
def compute_fuzzy_distance(range_dict, df):
    distances = []

    for _, row in df.iterrows():
        wine_distance = 0
        col_count = 0
        for col in df.columns:
            key = col.replace("_scld", "")
            if key in range_dict:
                value = row[col]
                q_low, q_high = range_dict[key]
                wine_distance += fuzzy_distance(value, q_low, q_high)
                col_count += 1
        
        distance = wine_distance / col_count if col_count > 0 else None # Cálculo del promedio de distancias
        distances.append(distance)

    distances = np.array(distances)

    return distances

In [4]:
# Funciones auxiliares para transformar df
def df_add_scaled_tastes(df, taste_features):
    copy_df = df.copy()
    # Scaling de sabores
    scaler = MinMaxScaler()
    scaled_tastes = [taste + "_scld" for taste in taste_features]
    copy_df[scaled_tastes] = scaler.fit_transform(copy_df[taste_features])
    return copy_df

def df_add_other_grapes(df, grapes, top_grapes):
    copy_df = df.copy()
    # Columna Otras Uvas
    other_grapes = [grape for grape in grapes if grape not in top_grapes]
    copy_df["Otras Uvas"] = copy_df[other_grapes].max(axis=1)
    return copy_df

def df_add_qual_price(df):
    copy_df = df.copy()
    # Zscore quality/price
    copy_df['rating_zscore'] = stats.zscore(copy_df["rating"])
    copy_df['price_zscore'] = stats.zscore(copy_df["price"])
    copy_df["zscore_quality_price"] = copy_df['rating_zscore'] - copy_df['price_zscore']
    qual_price_min = copy_df["zscore_quality_price"].min()
    qual_price_max = copy_df["zscore_quality_price"].max()
    copy_df["zscore_quality_price_scld"] = (copy_df["zscore_quality_price"] - qual_price_min)/(qual_price_max - qual_price_min)
    return copy_df

def df_add_user_similarity(df, scaled_tastes, user_tastes):
    copy_df = df.copy()
    # User similarity (Fuzzy Distance)
    distances = compute_fuzzy_distance(user_tastes, copy_df[scaled_tastes])
    copy_df["user_similarity"] = 1 - (distances / distances.max())
    return copy_df

def df_add_wine_score(df, user_input):
    copy_df = df.copy()
    # User Weights
    user_weights = user_input.get("weights")
    rating_w = user_weights.get("rating")
    price_quality_w = user_weights.get("price_quality")
    rating_qty_w = user_weights.get("rating_qty")
    user_similarity_w = user_weights.get("user_similarity")
    main_pairing_w = user_weights.get("main_pairing")
    
    # Puntaje sintético del vino
    copy_df["wine_score"] = (
        rating_w * (copy_df["rating"] / 5) + 
        price_quality_w * copy_df["zscore_quality_price_scld"] + 
        rating_qty_w * (copy_df["rating_qty"] / copy_df["rating_qty"].max()) +
        user_similarity_w * copy_df["user_similarity"] + 
        main_pairing_w * copy_df[user_input["main_pairing"]]
    )
    return copy_df

In [ ]:
def pick_wine(wines_df, user_input, top_n=20, d=.075):
    wines_copy = wines_df.copy()

    # User Inputs
    user_pairings = user_input.get("pairing_list")
    precio_min = user_input.get("precio_min")
    precio_max = user_input.get("precio_max")
    grape_list = user_input.get("grape_list")
    user_tastes = user_input.get("tastes")

    scaler = MinMaxScaler()

    # Filtro por pairings
    if user_pairings is not None:
        wine_base = wines_copy[wines_copy[user_pairings].sum(axis=1)>0]

    # Filtro por precio
    if precio_min is not None:
        wine_base = wine_base[wine_base["price"]>=precio_min]
    if precio_max is not None:
        wine_base = wine_base[wine_base["price"]<=precio_max]
    
    # Filtro por uvas
    if grape_list is not None:
        wine_base = wine_base[wine_base[grape_list].sum(axis=1)>0]

    # Zscore quality/price local
    wine_base = df_add_qual_price(wine_base)

    if len(wine_base) > 0:
        # User similarity (Fuzzy Distance)
        scaled_tastes = ["body_scld", "tannins_scld", "sweetness_scld", "acidity_scld"]
        wine_base = df_add_user_similarity(wine_base, scaled_tastes, user_tastes)

        # Puntaje sintético local del vino
        wine_base = df_add_wine_score(wine_base, user_input) # Puntaje "crudo"

        if len(wine_base) > max(top_n//2, 10):
            # Si el wine_base es lo suficientemente grande, el puntaje es escalado considerando la selección de filtros del usuario
            wine_base["wine_score_scld"] = scaler.fit_transform(wine_base[["wine_score"]])
        else:
            # Si el wine_base es pequeño, se considera el wine_score global (sin filtros de usuario)
            if wine_base.index.isin(wines_copy.index).all():
                wine_base["wine_score_scld"] = wines_copy.loc[wine_base.index]["wine_score_scld_gbl"]
            else:
                raise IndexError("Index in wine_base and global dataframe passed doesn't match!")


        # Selección de uno de los mejores top_n vinos
        top_wine_base = wine_base.nlargest(top_n, "wine_score")
        top_selection_weights = [(1 - d)**n for n in range(len(top_wine_base))] # Pesos decrecientes de a d%
        top_selection_weights = [w*(random.random() + 1) for w in top_selection_weights] # Agregamos factor aleatorio
        total_weight = sum(top_selection_weights)
        top_selection_weights = [w / total_weight for w in top_selection_weights] # Normalizamos para interpretabilidad
        selected_wine_id = int(random.choices(top_wine_base.index, top_selection_weights, k=1)[0])
        selected_wine = wine_base.loc[selected_wine_id]
        return selected_wine, top_wine_base
    else:
        logging.info("No wines match user input!")
        return None

In [27]:
def simulate_like(global_dataset: pd.DataFrame, selected_wine: pd.Series, global_score_name="wine_score_scld_gbl", local_score_name="wine_score_scld", alpha=.5, beta=.5, desv=.05):
    """
    Simula si al usuario le gustó o no el vino recomendado.


    Parámetros
    ---------
    global_dataset: pd.DataFrame
        el df que contiene el global score escalado
    selected_wine: pd.Series
        series con el wine seleccionado
    global_score_name: str
        nombre de la columna con el wine score global
    local_score_name: str
        nombre de la columna con el wine score local
    alpha: float
        multiplica la diferencia entre el score global y el local cuando es positiva (default 0.5)
    beta: float
        multiplica la diferencia entre el score global y el local cuando es negativa (default 0.5)
    desv: float
        desvío estandard del factor aleatorio con distribución normal que se le suma a la probabilidad de que le guste el vino al usuario (default 0.05)

    """
    global_score_norm = global_dataset.loc[selected_wine.name][global_score_name]
    local_score_norm = selected_wine[local_score_name]
    desvio_score = global_score_norm - local_score_norm
    ajuste = 0

    if desvio_score > 0: # El vino es mejor globalmente de lo que aparenta ser localmente
        ajuste = desvio_score * alpha
    elif desvio_score < 0: # El vino es peor globalmente de lo que aparenta ser localmente
        ajuste = desvio_score * beta
    else: # El vino es igual de bueno global y localmente
        ajuste = 0

    prob_like = local_score_norm + ajuste + np.random.normal(0, desv) # Score ajustado con factor random
    prob_like = min(max(prob_like, 0),.99)
    random_prob = random.random()
    liked = 0

    if random_prob < prob_like:
        liked = 1

    return liked, prob_like

### usuario

In [ ]:
wines_df = pd.read_csv("../../src/data/transformed/wines_clean.csv")
pairings = pd.read_csv("../../src/data/processed/aux/pairings.csv")
pairings = list(pairings["pairings"])
meals_df = pd.read_excel("../../src/data/raw/meals/Meals.xlsx")
grapes = pd.read_csv("../../src/data/processed/aux/grapes.csv")
grapes = grapes["grapes"]
grape_selection = wines_df[grapes]
scaler = MinMaxScaler()

taste_columns = ["body", "tannins", "sweetness", "acidity"]
scld_taste_columns = [taste + "_scld" for taste in taste_columns]

wines_copy = wines_df.copy()
wines_copy[scld_taste_columns] = scaler.fit_transform(wines_df[taste_columns])


# Creación de perfiles de sabor por pairing y obtención de sus desvíos para selección según distribución normal
profiles, deviations = build_pairing_profile(df=wines_copy, pairing_cols=pairings, taste_cols=scld_taste_columns)

# Elección aleatoria de una meal y obtención de sus pairings
selected_meal = select_random_meal(meals_df)
main_pairing = selected_meal["Ingrediente Principal"].lower()
pairing_list = selected_meal[2:] 
pairing_list = list(pairing_list.index[pairing_list>0])

# Elección del perfil del vino, teniendo en cuenta algo de sentido común de los usuarios
selected_profile = select_taste_profile(taste_cols=scld_taste_columns, pairing=main_pairing, profiles=profiles, deviations=deviations, prob=.2)

# Estaría bueno acá poder ver una distribución tipo histograma de los sabores según el main pairing elegido
selected_grapes = select_grapes(grape_selection, 8, 1, 4)

# Elección del rango de precios del vino, teniendo en cuenta lógica de usuario
selected_price_range = select_price_range(wines_df)

# Generación de user input
user_input = {
    "meal": selected_meal["Comida"],
    "main_pairing": main_pairing,
    "pairing_list": pairing_list,
    "grape_list": selected_grapes,
    "precio_min": selected_price_range[0],
    "precio_max": selected_price_range[1],
    "tastes": {
        "body": selected_profile["body_scld"][1],
        "tannins": selected_profile["tannins_scld"][1],
        "sweetness": selected_profile["sweetness_scld"][1],
        "acidity": selected_profile["acidity_scld"][1]
    },
    "weights": {
        "rating": .25,
        "price_quality": .25,
        "rating_qty": .1,
        "user_similarity": .3,
        "main_pairing": .1
    }
}

user_input

{'meal': 'Pechuga de Pollo Rellena',
 'main_pairing': 'poultry',
 'pairing_list': ['mild and soft cheese', 'poultry'],
 'grape_list': ['Cabernet Sauvignon'],
 'precio_min': 15,
 'precio_max': 20,
 'tastes': {'body': [np.float64(0.6917518248175184),
   np.float64(0.766131386861314)],
  'tannis': [np.float64(0.0), np.float64(0.3334678504170029)],
  'sweetness': [np.float64(0.45609698681732586), np.float64(1.0)],
  'acidity': [np.float64(0.5066863829240067), np.float64(0.6141185547126142)]},
 'weights': {'rating': 0.25,
  'price_quality': 0.25,
  'rating_qty': 0.1,
  'user_similarity': 0.3,
  'main_pairing': 0.1}}

In [ ]:
top_grapes = list(get_top_grapes(grape_selection, 8).index)

tra_wines = df_add_scaled_tastes(wines_df, ["body", "tannins", "sweetness", "acidity"])

tra_wines = df_add_other_grapes(tra_wines, grape_selection, top_grapes)

tra_wines = df_add_qual_price(tra_wines)

tra_wines = df_add_user_similarity(tra_wines, scld_taste_columns, user_input.get("tastes"))

tra_wines = df_add_wine_score(tra_wines, user_input)

scaler = MinMaxScaler()
tra_wines["wine_score_scld"] = scaler.fit_transform(tra_wines[["wine_score"]])

# Renombro las columnas con escala para saber que son a nivel global (todo el dataset) y no local (según filtros de usuario)
cols_to_rename = ['rating_zscore', 'price_zscore', 'zscore_quality_price', 'zscore_quality_price_scld', 'user_similarity', 'wine_score', 'wine_score_scld']
renamed_cols = [col_name + "_gbl" for col_name in cols_to_rename]
renaming_dict = dict(zip(cols_to_rename,renamed_cols))
tra_wines = tra_wines.rename(columns=renaming_dict)

selected_wine, top_wine_base = pick_wine(tra_wines, user_input, top_n=20)

if selected_wine is None:
    print("No encontramos vinos para tu preferencia")
else:
    display_columns = [
    "name", "year", "price", "rating",
    "rating_qty", "body_scld", "tannins_scld",
    "sweetness_scld", "acidity_scld", "rating_zscore",
    "price_zscore", "zscore_quality_price_scld", "user_similarity", "wine_score_scld_gbl", "wine_score_scld"]
    print(selected_wine[display_columns])

liked = simulate_like(tra_wines, selected_wine, alpha=0.3, beta=0.7)
liked

name                         Smoked Red Blend
year                                   2018.0
price                                   16.99
rating                                    3.9
rating_qty                             1163.0
body_scld                            0.715328
tannis_scld                          0.411891
sweetness_scld                       0.628766
acidity_scld                         0.565128
rating_zscore                        0.483771
price_zscore                        -0.685547
zscore_quality_price_scld            0.792406
user_similarity                      0.957897
wine_score_scld_gbl                  0.910095
wine_score_scld                           1.0
Name: 1813, dtype: object


(1, np.float64(0.9575592194652676))

In [9]:
top_wine_base

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,...,user_similarity_gbl,wine_score_gbl,wine_score_scld_gbl,rating_zscore,price_zscore,zscore_quality_price,zscore_quality_price_scld,user_similarity,wine_score,wine_score_scld
1813,https://www.vivino.com/dona-paula-smoked-red-b...,Smoked Red Blend,2018.0,Doña Paula,3.9,1163.0,16.99,0.6550,0.3628,0.2671,...,0.982333,0.825646,0.910095,0.483771,-0.685547,1.169318,0.792406,0.957897,0.794175,1.000000
1679,https://www.vivino.com/piattelli-cabernet-sauv...,Cabernet Sauvignon Reserve,2020.0,Piattelli,4.0,334.0,15.35,0.6436,0.5410,0.1567,...,0.906622,0.807377,0.870573,0.949171,-1.664878,2.614049,1.000000,0.777464,0.787175,0.981936
1607,https://www.vivino.com/clos-de-los-siete-clos-...,Clos de Los Siete (by Michel Rolland),2018.0,Clos de los Siete,4.0,8486.0,20.00,0.7262,0.4821,0.1994,...,0.887119,0.845812,0.953724,0.949171,1.111883,-0.162712,0.601005,0.730984,0.769547,0.936447
1608,https://www.vivino.com/clos-de-los-siete-clos-...,Clos de Los Siete (by Michel Rolland),2015.0,Clos de los Siete,4.0,8148.0,20.00,0.7262,0.4821,0.1994,...,0.887119,0.843944,0.949682,0.949171,1.111883,-0.162712,0.601005,0.730984,0.765563,0.926169
1676,https://www.vivino.com/altocedro-ano-cero-cabe...,Año Cero Cabernet Sauvignon,2019.0,Altocedro,4.0,335.0,15.99,0.6462,0.5442,0.1179,...,0.894026,0.803497,0.862180,0.949171,-1.282700,2.231871,0.945085,0.747446,0.764453,0.923303
377,https://www.vivino.com/clos-de-los-siete-clos-...,Clos de Los Siete (by Michel Rolland),2019.0,Clos de los Siete,4.0,7989.0,20.00,0.7262,0.4821,0.1994,...,0.887119,0.843065,0.947780,0.949171,1.111883,-0.162712,0.601005,0.730984,0.763690,0.921334
1609,https://www.vivino.com/clos-de-los-siete-clos-...,Clos de Los Siete (by Michel Rolland),2017.0,Clos de los Siete,4.0,7951.0,20.00,0.7262,0.4821,0.1994,...,0.887119,0.842855,0.947326,0.949171,1.111883,-0.162712,0.601005,0.730984,0.763242,0.920179
1589,https://www.vivino.com/finca-la-luz-callejon-d...,Callejon del Crimen Gran Reserva Cabernet Sauv...,2016.0,Finca La Luz,4.1,58.0,16.98,0.6385,0.5601,0.1276,...,0.895172,0.810899,0.878193,1.414571,-0.691519,2.106090,0.927011,0.750178,0.762489,0.918237
721,https://www.vivino.com/dona-paula-smoked-red-b...,Smoked Red Blend,2019.0,Doña Paula,3.8,282.0,16.99,0.6550,0.3628,0.2671,...,0.982333,0.812022,0.880621,0.018371,-0.685547,0.703918,0.725532,0.957897,0.762075,0.917168
732,https://www.vivino.com/piattelli-cabernet-sauv...,Cabernet Sauvignon Reserve,2019.0,Piattelli,3.8,188.0,15.35,0.6436,0.5410,0.1567,...,0.906622,0.789062,0.830950,0.018371,-1.664878,1.683249,0.866253,0.777464,0.742018,0.865411


In [10]:
list(top_wine_base.columns[-18:])

['tannis_scld',
 'sweetness_scld',
 'acidity_scld',
 'Otras Uvas',
 'rating_zscore_gbl',
 'price_zscore_gbl',
 'zscore_quality_price_gbl',
 'zscore_quality_price_scld_gbl',
 'user_similarity_gbl',
 'wine_score_gbl',
 'wine_score_scld_gbl',
 'rating_zscore',
 'price_zscore',
 'zscore_quality_price',
 'zscore_quality_price_scld',
 'user_similarity',
 'wine_score',
 'wine_score_scld']

In [11]:
tra_wines.loc[selected_wine.name]

wine_link                        https://www.vivino.com/altocedro-ano-cero-cabe...
name                                                   Año Cero Cabernet Sauvignon
year                                                                        2019.0
winery                                                                   Altocedro
rating                                                                         4.0
                                                       ...                        
zscore_quality_price_gbl                                                  0.412146
zscore_quality_price_scld_gbl                                             0.933752
user_similarity_gbl                                                       0.894026
wine_score_gbl                                                            0.803497
wine_score_scld_gbl                                                        0.86218
Name: 1676, Length: 130, dtype: object

In [12]:
tra_wines.columns[-11:]

Index(['tannis_scld', 'sweetness_scld', 'acidity_scld', 'Otras Uvas',
       'rating_zscore_gbl', 'price_zscore_gbl', 'zscore_quality_price_gbl',
       'zscore_quality_price_scld_gbl', 'user_similarity_gbl',
       'wine_score_gbl', 'wine_score_scld_gbl'],
      dtype='object')

In [ ]:
p = { 
    "body": selected_profile["body_scld"],
    "tannins": selected_profile["tannins_scld"],
    "sweetness": selected_profile["sweetness_scld"],
    "acidity": selected_profile["acidity_scld"]
}
p

{'body': ('marcado',
  [np.float64(0.6917518248175184), np.float64(0.766131386861314)]),
 'tannis': ('leve', [np.float64(0.0), np.float64(0.3334678504170029)]),
 'sweetness': ('intenso', [np.float64(0.45609698681732586), np.float64(1.0)]),
 'acidity': ('marcado',
  [np.float64(0.5066863829240067), np.float64(0.6141185547126142)])}

In [ ]:
tra_wines[tra_wines[main_pairing]==1][["body_scld","tannins_scld","sweetness_scld","acidity_scld"]].quantile([0, .25, .5, .75, 1])

,body_scld,tannis_scld,sweetness_scld,acidity_scld
0.00,0.204380,0.000000,0.000000,0.041018
0.25,0.544781,0.333468,0.247411,0.468561
0.50,0.691752,0.570218,0.342750,0.506686
0.75,0.766131,0.662228,0.456097,0.614119
1.00,1.000000,0.853914,1.000000,0.952681


In [15]:
scaler = MinMaxScaler()
tra_wines["wine_score_gbl_scld"] = scaler.fit_transform(tra_wines[["wine_score_gbl"]])



In [16]:
selected_wine["wine_score"]

np.float64(0.7644525594662244)

In [ ]:
# Generar clase
# Crear datos sintéticos